In [1]:
import os
import psycopg
import numpy as np
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import ollama

# Charger les variables d'environnement
load_dotenv("../src/.env")

# Connexion PostgreSQL
conn = psycopg.connect(
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)

print("Connexion PostgreSQL OK")


c:\Users\Lenovo\Chatbot-Rag\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Connexion PostgreSQL OK


In [2]:
def create_conversation_list(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.read().split("\n")
        return [l.strip() for l in lines if l.strip()]

corpus_list = create_conversation_list("../data/corpus.txt")
len(corpus_list)


1982

In [3]:
with conn.cursor() as cur:
    cur.execute("DROP TABLE IF EXISTS embeddings;")
    cur.execute("""
        CREATE TABLE embeddings (
            id SERIAL PRIMARY KEY,
            corpus TEXT,
            embedding FLOAT8[]
        );
    """)
conn.commit()

print("Table embeddings créée ✔")


Table embeddings créée ✔


In [4]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def get_embedding(text):
    emb = model.encode(text)
    return emb.tolist()


In [5]:
with conn.cursor() as cur:
    for text in corpus_list:
        emb = get_embedding(text)
        cur.execute(
            "INSERT INTO embeddings (corpus, embedding) VALUES (%s, %s)",
            (text, emb)
        )
    conn.commit()

print("Corpus + embeddings insérés ✔")


Corpus + embeddings insérés ✔


In [6]:
def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [7]:
def similar_corpus(input_text, top_k=3):
    query_emb = get_embedding(input_text)

    with conn.cursor() as cur:
        cur.execute("SELECT id, corpus, embedding FROM embeddings")
        rows = cur.fetchall()

    results = []
    for id_, corpus, emb in rows:
        score = cosine_similarity(query_emb, emb)
        results.append((id_, corpus, score))

    results.sort(key=lambda x: x[2], reverse=True)
    return results[:top_k]


In [8]:
def build_context(similar_results):
    return "\n".join([f"- {corpus}" for _, corpus, _ in similar_results])


In [9]:
def generate_answer(question):
    similar = similar_corpus(question, top_k=3)
    context = build_context(similar)

    prompt = f"""
Tu es un assistant UBS.

Contexte :
{context}

Question :
{question}

Réponse (basée UNIQUEMENT sur le contexte) :
"""
    
    response = ollama.generate(
        model="phi3:mini",
        prompt=prompt
    )

    return response["response"]


In [10]:
generate_answer("Comment ouvrir un dossier à l'UBS ?")


"Pour ouvrir un dossier de candidature pour obtenir le diplôme en droit des affaires et droits du personnel chez UBS, il vous faudra contacter directement votre superviseur ou le service clientèle spécialisé dans les admissions à l'UBS. Il est impératif que ces dossiers soient préparés selon les normes établies par la banque et qu’ils reflètent correctement vos compétences et objectifs professionnels."